In [2]:
import copy

############ DATASET MAKING #######################

import random

total_l_limit=0
students=[]
projects=[]
lecturers=[]
lecturers_offering={}
project_lecturer={}
lecturers_limit={}
s_pref={}

for n in range(140):
    students.append('s'+str(n+1))
    
for n in range(41):
    projects.append('p'+str(n+1))

for n in range(41):
    lecturers.append('l'+str(n+1))

lcnt=0
for l in lecturers:
    lecturers_offering[l]=[projects[lcnt]]
    lcnt+=1
    
pcnt=0
for p in projects:
    project_lecturer[p]=[lecturers[pcnt]]
    pcnt+=1
    
for l in lecturers:
    rnd = random.randint(0,1)
    if rnd==0:
        lecturers_limit[l]=4
        total_l_limit+=4
    if rnd==1:
        lecturers_limit[l]=8
        total_l_limit+=8

backup_ll=copy.copy(lecturers_limit)
for sd in students:
    rnd1 = random.randint(1,41)
    
    rnd2 = copy.copy(rnd1)
    while rnd2==rnd1:
        rnd2 = random.randint(1,41)
    
    rnd3 = copy.copy(rnd2)
    while rnd3==rnd2 or rnd3==rnd1:
        rnd3 = random.randint(1,41)
        
    rnd4 = copy.copy(rnd3)
    while rnd4==rnd3 or rnd4==rnd2 or rnd4==rnd1:
        rnd4 = random.randint(1,41)
    
    s_pref[sd]=['p'+str(rnd1),'p'+str(rnd2),'p'+str(rnd3),'p'+str(rnd4)]

###############################################################################

############################ PROJECT SCORE/RANK CALCULATION #####################
proj_score={}
for stud in s_pref:
    left_proj=copy.copy(projects)
    for i,proj in enumerate(s_pref[stud]):
        if proj not in proj_score:
            proj_score[proj]=0
        if proj in proj_score:
            proj_score[proj]+=i+1
            left_proj.remove(proj)
    for project in left_proj:
        if project not in proj_score:
            proj_score[project]=0
        proj_score[project]+=4
proj_score_updated=sorted(proj_score.items(), key = lambda kv:(kv[1], kv[0]),reverse=False)
# print(proj_score_updated)

#################################################################################

############################ ALLOCATING STUDENTS ACCORDING TO THEIR PREFERENCE ####################
done_sd=[]
left_sd=copy.copy(students)
full_assign_info={}

for i in range(4):
    for ind,pscore in enumerate(proj_score_updated):
        pop_proj=pscore[0]
        pop_proj_stud=[]
        pop_proj_lec=[]
        for stud in s_pref:
            if s_pref[stud][i]==pop_proj:
                pop_proj_stud.append(stud)

        for lec in lecturers_offering:
            if pop_proj in lecturers_offering[lec]:
                pop_proj_lec.append(lec)

        for lec in pop_proj_lec:
            cap=lecturers_limit[lec]
            for sd in pop_proj_stud:
                if (lecturers_limit[lec]>0) and (sd not in done_sd):
                    full_assign_info[sd]=[lec,pop_proj]
                    lecturers_limit[lec]-=1
                    done_sd.append(sd)
                    left_sd.remove(sd)
                    
####################################################################################################

########################### FIND SPARE SEAT PROJECTS ###############################################
                    
spare_seat_projects={}
for lect in lecturers_limit:
    if lecturers_limit[lect]>0:
        for proj in lecturers_offering[lect]:
            spare_seat_projects[proj]=lect  
# print(spare_seat_projects)

####################################################################################################

################ RE-ALLOCATION OF ALLOCATED STUDENTS AND ALLOCATION OF UNALLOCATED STUDENTS ###############

temp_left_sd=copy.copy(left_sd)
for everysd in temp_left_sd:
    donesd=False
    for everysd_pref in range(4):
        for n in range(4):
            for sd in s_pref:
                if sd in full_assign_info:
                    if s_pref[sd][n] in spare_seat_projects and full_assign_info[sd][1]==s_pref[everysd][everysd_pref] and sd not in left_sd and s_pref[sd][n]!=full_assign_info[sd][1]:
                        assigned_lec=full_assign_info[sd][0]
                        assigned_proj=full_assign_info[sd][1]
                        full_assign_info[sd][0]=spare_seat_projects[s_pref[sd][n]]
                        full_assign_info[sd][1]=s_pref[sd][n]
                        full_assign_info[everysd]=[assigned_lec,assigned_proj]
                        lecturers_limit[spare_seat_projects[s_pref[sd][n]]]-=1
                        done_sd.append(everysd)
                        left_sd.remove(everysd)
                        if lecturers_limit[spare_seat_projects[s_pref[sd][n]]]==0:
                            del spare_seat_projects[s_pref[sd][n]]
                        donesd=True
                        break
            if donesd==True:
                break
        if donesd==True:
            donesd=False
            break  
#print(full_assign_info)

#########################################################################################################

####################### MAKING INFO, WHICH STUDENTS ARE ALLOCATED TO WHICH LECTURER #####################

stud_lec_assignment={}
for assign in full_assign_info:
    if full_assign_info[assign][0] not in stud_lec_assignment:
        stud_lec_assignment[full_assign_info[assign][0]]=[]
    stud_lec_assignment[full_assign_info[assign][0]].append(assign)
# print(stud_lec_assignment)

#########################################################################################################

####################### DIVIDE INTO PERFECT,UNBALANCED AND SPARE SEAT GROUPS ############################

unbalanced_group={}
spare_seat_group={}
stud_lec_assignment_temp=copy.copy(stud_lec_assignment)
for l in stud_lec_assignment_temp:
    if len(stud_lec_assignment[l])%3!=0 and len(stud_lec_assignment[l])%4!=0 and len(stud_lec_assignment[l])%4!=3:
        unbalanced_group[l]=stud_lec_assignment[l]
        del stud_lec_assignment[l]
    elif len(stud_lec_assignment[l])%3==0 or len(stud_lec_assignment[l])%4==3:
        spare_seat_group[l]=stud_lec_assignment[l]
        del stud_lec_assignment[l]
# print(len(stud_lec_assignment))  #perfect groups people
# print(len(unbalanced_group))
# print(len(spare_seat_group))

#########################################################################################################

############ DIVIDE UNBALANCED INTO GIVER,TAKER GROUP TO ADJUST THEMSELVES TO MAKE PERFECT GROUPS ############

giver={}
taker={}
countd=0
for lec in unbalanced_group:
    if len(unbalanced_group[lec])==5 or len(unbalanced_group[lec])==1:
        giver[lec]=unbalanced_group[lec]
        countd+=1
    if len(unbalanced_group[lec])==2:
        taker[lec]=unbalanced_group[lec]
        countd+=1       
# print(giver)
# print('\n')
# print(taker)

############################################################################################################

################## LOOP THROUGH GIVER TO ASSIGN STUDENTS TO TAKER, TO MAKE PERFECT GROUPS ##################

groupdone=False
givertemp=copy.copy(giver)
takertemp=copy.copy(taker)
donetaker=[]
for lecg in givertemp:
    groupdone=False
    for stud in givertemp[lecg]:
        for lect in takertemp:
            if lecturers_offering[lect][0] in s_pref[stud] and lect not in donetaker:
                donetaker.append(lect)
                taker[lect].append(stud)
                giver[lecg].remove(stud)
                stud_lec_assignment[lect]=taker[lect]
                stud_lec_assignment[lecg]=giver[lecg]
                lecturers_limit[lecg]+=1
                lecturers_limit[lect]-=1
                del giver[lecg]
                del taker[lect]
                del unbalanced_group[lecg]
                del unbalanced_group[lect]
                groupdone=True
                break
        if groupdone==True:
            groupdone=False
            break
        
###########################################################################################################

#################### MAKE SPARE SEAT META, TO FIND LIMITS OF EVERY SPARE SEAT GROUPS ######################

spare_seat_group_meta={}
for lec in spare_seat_group:
    limit=lecturers_limit[lec]
    if limit>0:
        if limit==5:
            spare_seat_group_meta[lecturers_offering[lec][0]]=1
        else:
            spare_seat_group_meta[lecturers_offering[lec][0]]=limit

###########################################################################################################

######################## FIND GROUPS FOR GIVER, TAKER STUDENTS IN SPARE SEAT GROUPS #######################

groupdone2=False
giverdone2=False
takerdone2=False
givertemp2=copy.copy(giver)
takertemp2=copy.copy(taker)
donetaker2=[]
donegiver2=[]

for lec in givertemp2:
    for stud in givertemp2[lec]:
        for proj in spare_seat_group_meta:
            if proj in s_pref[stud] and spare_seat_group_meta[proj]>0:
                spare_seat_group_meta[proj]-=1
                proj_lecturer=project_lecturer[proj][0]
                spare_seat_group[proj_lecturer].append(stud)
                giver[lec].remove(stud)
                if len(giver[lec])==0:
                    del giver[lec]
                elif len(giver[lec])==3 or len(giver[lec])==4 or len(giver[lec])==6 or len(giver[lec])==7 or len(giver[lec])==8:
                    spare_seat_group[lec]=giver[lec]
                    del giver[lec]
                    giverdone2=True
                break
        if giverdone2==True:
            giverdone2=False
            break
    
for lec in takertemp2:
    for stud in takertemp2[lec]:
        for proj in spare_seat_group_meta:
            if proj in s_pref[stud] and spare_seat_group_meta[proj]>0:
                spare_seat_group_meta[proj]-=1
                proj_lecturer=project_lecturer[proj][0]
                spare_seat_group[proj_lecturer].append(stud)
                taker[lec].remove(stud)
                if len(taker[lec])==0:
                    del taker[lec]
                elif len(taker[lec])==3 or len(taker[lec])==4 or len(taker[lec])==6 or len(taker[lec])==7 or len(taker[lec])==8:
                    spare_seat_group[lec]=taker[lec]
                    del taker[lec]
                    takerdone2=True
                break
        if takerdone2==True:
            takerdone2=False
            break
# print(giver,'\n')
# print(taker,'\n')

############################################################################################################

############################# MERGE PERFECT AND SPARE SEAT, MERGE GIVER AND TAKER ##########################

temp_perfect_group={**stud_lec_assignment, **spare_seat_group}
giver_taker_group={**giver, **taker}

############################################################################################################

##### FIND GROUPS IN PERFECT GROUPS FOR REST OF THE GIVER TAKER STUDENTS ACCORDING TO THEIR PREFERENCES #####

err=[1,2,5]            

left_stud_givertaker=[]
for l in giver_taker_group:
    for s in giver_taker_group[l]:
        left_stud_givertaker.append(s)
        
ll_for_perfect_group={}
ll_pgroup={}
done_studs=[]
for l in temp_perfect_group:
    ll_for_perfect_group[l]=backup_ll[l]-len(temp_perfect_group[l])
    if ll_for_perfect_group[l]==4:
        ll_pgroup[l]=[2,3]
    elif ll_for_perfect_group[l]==5:
        ll_pgroup[l]=[1,1] 
    elif ll_for_perfect_group[l]==8:
        ll_pgroup[l]=[3,4]
    else:
        ll_pgroup[l]=[ll_for_perfect_group[l],ll_for_perfect_group[l]]
    
    if ll_pgroup[l][0]<=len(left_stud_givertaker) and ll_pgroup[l][0]>0 and len(left_stud_givertaker)>0:
        can_cover_upto=ll_pgroup[l][1]
        stud_cnt=0
        temp_lstud=[]
        for s in left_stud_givertaker:
            if lecturers_offering[l][0] in s_pref[s]:
                temp_lstud.append(s)
        if len(temp_lstud)>=ll_pgroup[l][0]:
            for s in temp_lstud:
                if stud_cnt<=can_cover_upto:
                    temp_perfect_group[l].append(s)
                    left_stud_givertaker.remove(s)
                    stud_cnt+=1
                    if stud_cnt==can_cover_upto:
                        break
                        
#############################################################################################################

### FIND GROUPS IN PERFECT GROUPS FOR REST OF THE GIVER TAKER (WITHOUT ANY PREFERENCE, ANY AVAILABLE SEAT) ###
err=[1,2,5]
                    
for l in temp_perfect_group:
    ll_for_perfect_group[l]=backup_ll[l]-len(temp_perfect_group[l])
    if ll_for_perfect_group[l]==4 and len(temp_perfect_group[l])!=0:
        ll_pgroup[l]=[2,4]
    elif ll_for_perfect_group[l]==4 and len(temp_perfect_group[l])==0:
        ll_pgroup[l]=[3,4]
    elif ll_for_perfect_group[l]==5:
        ll_pgroup[l]=[1,1] 
    elif ll_for_perfect_group[l]==8:
        ll_pgroup[l]=[3,4]
    else:
        ll_pgroup[l]=[ll_for_perfect_group[l],ll_for_perfect_group[l]]
        
    if ll_pgroup[l][0]<=len(left_stud_givertaker) and ll_pgroup[l][0]>0 and len(left_stud_givertaker)>0:
        can_cover_upto=ll_pgroup[l][1]
        stud_cnt=0
        templsgt=copy.copy(left_stud_givertaker)
        for s in templsgt:
            if stud_cnt<=can_cover_upto and s in left_stud_givertaker:
                temp_perfect_group[l].append(s)
                left_stud_givertaker.remove(s)
                stud_cnt+=1
                if stud_cnt==can_cover_upto:
                    break
    
##########################################################################################################

################# PRINT COVERED STUDENTS, AND ASSIGNED LECTURERS : TOTAL STUDENTS 140 #####################

counts=0
for l in temp_perfect_group:
    for s in temp_perfect_group[l]:
        counts+=1
print('COVERED STUDENTS:',counts)
print('LECTURERS-STUDENTS ALLOCATIONS: ',temp_perfect_group)

###########################################################################################################

COVERED STUDENTS: 140
LECTURERS-STUDENTS ALLOCATIONS:  {'l27': ['s16', 's35', 's80', 's87'], 'l21': ['s20', 's31', 's78', 's93', 's69', 's15'], 'l19': ['s18', 's22', 's27', 's60'], 'l17': ['s11', 's100', 's132', 's140'], 'l6': ['s7', 's13', 's23', 's24'], 'l18': ['s25', 's103', 's124', 's107'], 'l2': ['s36', 's44', 's79', 's45'], 'l23': ['s2', 's46', 's94', 's110'], 'l28': ['s41', 's43', 's65', 's76'], 'l9': ['s9', 's114', 's117', 's126'], 'l13': ['s33', 's53', 's86', 's138'], 'l40': ['s48', 's95', 's106', 's121'], 'l32': ['s10', 's118', 's133', 's58'], 'l10': ['s52', 's57', 's111', 's116'], 'l3': ['s14', 's72', 's85', 's131'], 'l37': ['s88', 's134', 's136', 's115'], 'l24': ['s113', 's102', 's104', 's63'], 'l25': ['s38', 's75', 's109', 's130', 's81', 's119'], 'l35': ['s96', 's137', 's21'], 'l15': ['s29', 's59', 's77', 's101'], 'l39': ['s55', 's67', 's1', 's42'], 'l33': [], 'l20': ['s32', 's50', 's4'], 'l36': [], 'l16': ['s12', 's39', 's64', 's91', 's97', 's99', 's128'], 'l5': ['s30', '

In [3]:
####################### TO EVALUATE ALGORITHM, PREFERENCE DETAILS AND GRAPH ######################

import pandas as pd
import plotly.express as px

def findrank(stud,proj):
    cnt=0
    for projects in s_pref[stud]:
        cnt+=1
        if projects==proj:
            return cnt
    return 5
    
pref_df=pd.DataFrame(columns=['pref','freq'])
first=0
second=0
third=0
fourth=0
fifth=0
rank_score=0
for l in temp_perfect_group:
    for sd in temp_perfect_group[l]:
        rank=findrank(sd,lecturers_offering[l][0])
        if rank==1:
            first+=1
        elif rank==2:
            second+=1
        elif rank==3:
            third+=1
        elif rank==4:
            fourth+=1
        elif rank==5:
            fifth+=1

        rank_score+=rank

pref_df.loc[len(pref_df)]=['First pref',first]
pref_df.loc[len(pref_df)]=['Second pref',second]
pref_df.loc[len(pref_df)]=['Third pref',third]
pref_df.loc[len(pref_df)]=['Fourth pref',fourth]
pref_df.loc[len(pref_df)]=['Fifth pref',fifth]

print(pref_df)
fig = px.bar(pref_df, x='pref', y='freq',labels={
                     "pref": "Preference",
                     "freq": "Frequency"},height=300, width=500)
fig.update_xaxes(dtick=1)
fig.show()
fig.write_image("./pref_result.jpeg", scale=1)

############################################################################################################

          pref freq
0   First pref  114
1  Second pref   14
2   Third pref    8
3  Fourth pref    3
4   Fifth pref    1
